In [ ]:

# Importing libraries 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential , Model , load_model
from tensorflow.keras.applications.vgg16 import VGG16   # importing vgg16 model for transfered learning
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout , Input, Lambda
from tensorflow.keras.optimizers import Adam  
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os      
import random
import matplotlib.pyplot as plt

In [ ]:
data_path='gdrive/My Drive/TB09'   # Give path of  your dataset folder 
os.chdir(data_path)         # Chaging path to data_path

In [ ]:
# Folders name of train ,test , valid data

train_path="train"     
test_path="test" 
valid_path="val" 
image_size = [224,224]   # targert image size

In [ ]:
vgg = VGG16(input_shape=image_size+[3],weights='imagenet',include_top=False)  # Defining vgg16 model and parameter 
# Why to using vgg16 model ?
#The model achieves 92.7% top-5 test accuracy in ImageNet, which is a dataset of over 14 million images belonging to 1000 classes
#VGG16 was trained for weeks and was using NVIDIA Titan Black GPU’s

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
for layers in vgg.layers:  # As vgg16 is trained on large dataset and with a lot of gpu power 
  layers.trainable = False # we dont need to train the whole model again 

In [ ]:
folder = os.listdir('train')

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
prediction=Dense(len(folder), activation='softmax')(x) 

In [ ]:
model = Model(inputs=vgg.input ,outputs=prediction)  # Preparing model

In [ ]:
model.summary() # How oue model look like

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
# Compiling model with Adam optimizer
model.compile(loss="categorical_crossentropy",
              optimizer='Adam',
              metrics=['accuracy'])

In [ ]:
# As for images we need to preprocess them before using them to train  our model
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), classes=['crack', 'not_crack'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224), classes=['crack', 'not_crack'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224), classes=['crack', 'not_crack'], batch_size=10, shuffle=False)

Found 32000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [ ]:
r =  model.fit(x=train_batches, validation_data=valid_batches, epochs=5, steps_per_epoch=180, validation_steps=10) # Training model on training dataset
'''
or if you have a dedicated gpu , you can use it to train your model
with tf.device('/gpu:0'):
  r =  model.fit(x=train_batches, validation_data=valid_batches, epochs=5, steps_per_epoch=180, validation_steps=10)
'''